# ライブラリのインストール

In [1]:
!pip install optuna -q
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
import time
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 19.0 MB/s eta 0:00:00


# データ読み込み

In [2]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/github/利益予測データベース.xlsx'
df = pd.read_excel(file_path)
data = df.values

Mounted at /content/drive
           sample  group      ni_t     ni_t1     ni_t2     ni_t3     ni_t4  \
0  201003N0000001      1  0.053142  0.077657  0.073253  0.097867  0.098210   
1  200903N0000001      1  0.083942  0.079182  0.105787  0.106157  0.073416   
2  200803N0000001      1  0.052085  0.069586  0.069830  0.048293  0.043422   
3  200703N0000001      1  0.060600  0.060812  0.042056  0.037814  0.033996   
4  200603N0000001      1  0.080197  0.055463  0.049869  0.044834  0.040998   

      ni_t5   cy_e_t1  cy_sale_t1  ...  cy_iva_c_t1  cy_intan_c_t1  \
0  0.067920  0.077657    7.220336  ...    -0.025984       0.009102   
1  0.066011  0.079182    7.815907  ...    -0.073469       0.002539   
2  0.039038  0.069586    5.465569  ...    -0.083399       0.000139   
3  0.031088  0.060812    4.632824  ...     0.105868       0.005575   
4  0.037042  0.055463    6.099235  ...    -0.012547       0.000679   

   cy_ap_c_t1  cy_dlc_c_t1  cy_txp_c_t1  cy_dltt_c_t1  cy_re_c_t1  \
0    0.008368  

# 訓練データとテストと検証データに分割

In [10]:
X = df.iloc[:, 3:8].values
y = df.iloc[:, 2].values

In [11]:
# Optunaでの評価には検証データを、最終的なモデル性能の評価にはテストデータを使用する
from sklearn.model_selection import train_test_split
# まず、データ全体を「訓練＋検証データ」(80%)と「テストデータ」(20%)に分割します。
# テストデータは最終評価まで一切使用しません。
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 次に、上で作成した「訓練＋検証データ」を「訓練データ」と「検証データ」に分割します。
# この例では、80%のデータをさらに80%（訓練用）と20%（検証用）に分けています。
# これにより、全体に対して 訓練: 64%, 検証: 16%, テスト: 20% の比率になります。
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# 各データセットの形状（shape）を出力して、分割が正しく行われたか確認します
print(f"訓練データ (X_train) の形状: {X_train.shape}")
print(f"検証データ (X_val) の形状:   {X_val.shape}")
print(f"テストデータ (X_test) の形状:  {X_test.shape}")
print("-" * 40)
print(f"訓練データ (y_train) の形状: {y_train.shape}")
print(f"検証データ (y_val) の形状:   {y_val.shape}")
print(f"テストデータ (y_test) の形状:  {y_test.shape}")

訓練データ (X_train) の形状: (32652, 5)
検証データ (X_val) の形状:   (8163, 5)
テストデータ (X_test) の形状:  (10204, 5)
----------------------------------------
訓練データ (y_train) の形状: (32652,)
検証データ (y_val) の形状:   (8163,)
テストデータ (y_test) の形状:  (10204,)


# Optuna 目的関数定義

| 代码                          | 作用（日语）       | 用法例子           |
| --------------------------- | ------------ | -------------- |
| `trial.suggest_categorical` | 候補リストから選ぶ    | `[64, 128]` など |
| `trial.suggest_int`         | 整数を範囲から選ぶ    | `2, 4`         |
| `trial.suggest_float`       | 実数を範囲から選ぶ    | `0.0, 0.2`     |
| `trial.suggest_float`  | 対数スケールの実数を選ぶ | `1e-4, 1e-1, log=True`   |

In [15]:
def objective(trial):
    # Optunaでハイパーパラメータ自動探索
    params = {
        'objective': 'regression_l1',  # MAE回帰
        'metric': 'mae',
        'n_estimators': trial.suggest_int('n_estimators', 10, 10000, step=50),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.05),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0, step=0.05),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        'random_state': 42,
        'n_jobs': -1,
        'verbose': -1
    }
    model = lgb.LGBMRegressor(**params) # Dictionary Unpacking
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)
    return mae  # MAEが小さいほど良い

 lgb.LGBMRegressor(**params)
    # `**` を付けることで、params辞書が展開（アンパック）され、
    # 中のキーと値がそれぞれキーワード引数(ひきすう)としてモデルに渡されます。
    # つまり、以下のようにコードを書いたのと同じ意味になります。
    # lgb.LGBMRegressor(objective='regression_l1',
    #                   metric='mae',
    #                   n_estimators=100,
    #                   learning_rate=0.05,
    #                   ...)
    model = lgb.LGBMRegressor(**params)

| 参数                  | 中文说明               | 日语说明                | 常见调节建议                   |
| ------------------- | ------------------ | ------------------- | ------------------------ |
| objective           | 回归/损失函数（MAE，绝对值损失） | 目的関数/損失関数（MAE，絶対誤差） | 固定为回归问题时用 regression\_l1 |
| metric              | 评估指标（平均绝对误差）       | 評価指標（MAE）           | 固定                       |
| n\_estimators       | 树的棵数               | 決定木の本数              | 100-1000，树越多训练越慢         |
| learning\_rate      | 学习率                | 学習率                 | 0.005-0.1，越小越稳健          |
| max\_depth          | 最大树深               | 木の深さの最大値            | 3-10，越大模型越复杂             |
| num\_leaves         | 叶子数                | 葉の数                 | 20-150，越大表达力强，易过拟合       |
| min\_child\_samples | 叶节点最小样本数           | 葉の最小データ数            | 防止叶节点太小，5-50             |
| subsample           | 行采样比例              | データサンプリング率          | 0.6-1.0，防止过拟合            |
| colsample\_bytree   | 列采样比例              | 特徴量サンプリング率          | 0.6-1.0，防止过拟合            |
| reg\_alpha          | L1正则               | L1正則化項              | 大一些会让模型稀疏化，减少过拟合         |
| reg\_lambda         | L2正则               | L2正則化項              | 控制参数大小，减少过拟合             |
| random\_state       | 随机种子               | 乱数シード               | 固定用于复现                   |
| n\_jobs             | 并行数                | 並列計算数               | -1使用全部CPU                |
| verbose             | 日志详细度              | ログの出力レベル            | -1 静默                    |


# Optuna実行

In [16]:
import plotly.io as pio
# Colab環境でプロットを正しく表示するための設定です
pio.renderers.default = 'colab'

# ステップ1: studyオブジェクトを作成します。
# 私たちの目標はMAE（平均絶対誤差）を最小にすることなので、direction='minimize'と設定します。
study = optuna.create_study(direction='minimize')

# ステップ2: 最適化を実行します。
# objective関数を30回試行(n_trials=50)して、最適なハイパーパラメータを探します。
# この処理には数分かかることがあります。
print("optunaによるハイパーパラメータ探索を開始します...")
study.optimize(objective, n_trials=50)


# 探索完了後に結果を出力します。
print("\n探索が完了しました。")
print(f"最良スコア (検証MAE): {study.best_value}")
print("最良のハイパーパラメータ:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# ステップ3: 結果を可視化します。
from optuna.visualization import plot_optimization_history, plot_param_importances


print("\n--------------------------- 可視化結果 ------------------------------")

# 探索履歴のプロット
# 各試行でMAEがどのように改善されていったかを確認できます。
fig1 = plot_optimization_history(study)
fig1.show()


# ハイパーパラメータの重要度のプロット
# どのパラメータが最終的なスコアに最も影響を与えたかを確認できます。
fig2 = plot_param_importances(study)
fig2.show()


[I 2025-06-30 06:45:12,774] A new study created in memory with name: no-name-765977cf-c24f-4343-83e6-271079dffcf2
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:702: UserWarning:

The distribution is specified by [10, 10000] and step=50, but the range is not divisible by `step`. It will be replaced by [10, 9960].

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



optunaによるハイパーパラメータ探索を開始します...


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

[I 2025-06-30 06:45:19,586] Trial 0 finished with value: 0.05733504500019495 and parameters: {'n_estimators': 2010, 'learning_rate': 0.09431683251912547, 'max_depth': 6, 'num_leaves': 124, 'min_child_samples': 43, 'subsample': 0.6, 'colsample_bytree': 0.9, 'reg_alpha': 0.009663517493534383, 'reg_lambda': 7.049432252785922e-06}. Best is trial 0 with value: 0.05733504500019495.
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:702: UserWarning:

The distribution is specified by [10, 10000] and step=50, but the range is not divisible by `step`. It will be replaced by [10, 9960].

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sk


探索が完了しました。
最良スコア (検証MAE): 0.05617066447585539
最良のハイパーパラメータ:
  n_estimators: 410
  learning_rate: 0.010175309778890806
  max_depth: 9
  num_leaves: 39
  min_child_samples: 10
  subsample: 0.9
  colsample_bytree: 0.7
  reg_alpha: 0.023993812648638396
  reg_lambda: 0.09904534086076029

--------------------------- 可視化結果 ------------------------------


# 最良パラメータで再学習＆評価

In [17]:
print("--- ステップ4: 最終モデルの訓練と評価を開始します ---")

# 1. Optunaで見つけた最良のハイパーパラメータを取得します
best_params = study.best_params  # ディクショナリ
print("\n取得した最良のハイパーパラメータ:")
print(best_params)

# 2. LightGBMの固定パラメータと合体（metricやobjective、random_state等は明示的に追加！）
final_model_params = {
    'objective': 'regression_l1',
    'metric': 'mae',
    'random_state': 42,
    'n_jobs': -1,
    'verbose': -1
}
final_model_params.update(best_params)  # best_paramsの内容で上書き

# 3. モデル再学習
final_model = lgb.LGBMRegressor(**final_model_params)
final_model.fit(X_train_val,y_train_val)

# 4. テストデータで予測＆評価
final_pred = final_model.predict(X_test)
final_mae = mean_absolute_error(y_test, final_pred)

print("\n--- 再学習したモデルの評価結果 ---")
print(f"最終MAE: {final_mae:.6f}")

--- ステップ4: 最終モデルの訓練と評価を開始します ---

取得した最良のハイパーパラメータ:
{'n_estimators': 410, 'learning_rate': 0.010175309778890806, 'max_depth': 9, 'num_leaves': 39, 'min_child_samples': 10, 'subsample': 0.9, 'colsample_bytree': 0.7, 'reg_alpha': 0.023993812648638396, 'reg_lambda': 0.09904534086076029}


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.




--- 再学習したモデルの評価結果 ---
最終MAE: 0.057437
